In [ ]:
! pip install transformers datasets
! pip install evaluate
! pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl 

### Προσοχή

Μη διαγράψετε τα # insert your code here σχόλια, καθώς βοηθούν στη διόρθωση. Συμπληρώστε τον κώδικά σας μετά από τα σχόλια αυτά.

# Μέρος Α: Fine-tune a pretrained model

Τα γλωσσικά μοντέλα αποτελούνται από δύο στάδια εκπαίδευσης:
1. Pre-training σε μεγάλα unlabelled datasets. Το pre-training είναι υπολογιστικά πολύ ακριβό και γι αυτό στην πράξη δε το χρησιμοποιούμε όταν θέλουμε να τρέξουμε ένα μοντέλο σε ένα καινούργιο dataset. Μπορούμε να σκεφτούμε το pre-training ως τη διαδικασία εκμάθησης γλωσσικών κανόνων κι εννοιών, οι οποίες στη συνέχεια μπορούν να χρησιμοποιηθούν για διάφορους σκοπούς.

2. Fine-tuning σε μικρότερα labelled datasets. Το fine-tuning πρακτικά εκμεταλλεύεται τις ιδιότητες του transfer learning προκειμένου να μεταφέρουμε τη 'γνώση' που έχει αποθηκευθεί στο γλωσσικό μοντέλο κατά τη διάρκεια του pre-training σε συγκεκριμένα task. Κάθε task εξυπηρετείται μέσω στοχευμένων datasets. Για παράδειγμα, κάποια datasets αναφέρονται στην ταξινόμηση κειμένων σε κατηγιορίες (text classification), άλλα datasets περιέχουν ερωτήσεις οι οποίες πρέπει να απαντηθούν (question answering) κι άλλα πολλά.

Κάποια κλασικά tasks της επεξεργασίας φυσικής γλώσσας είναι τα ακόλουθα:
- Text classification
- Question answering
- Natural language inference
- Fill mask
- Semantic similarity

Περισσότερες πληροφορίες μπορείτε να βρείτε στον ακόλουθο σύνδεσμο στο domain Natural Language Processing: https://huggingface.co/models

Στο πρώτο κομμάτι της παρούσας εργαστηριακής άσκησης, θα χρησιμοποιήσουμε το pre-training fine-tuning σενάριο για να ταξινομήσουμε reviews.

## Pipelines

Με τη χρήση του text-classification pipeline μπορούμε να τρέξουμε γλωσσικά μοντέλα που αφορούν tasks ταξινόμησης. Το natural language inference (NLI) task αποτελεί ένα task ταξινόμησης, αφού το σχετικό μοντέλο (εν προκειμένω το roberta-large-mnli) καλείται να ταξινομήσει ένα κείμενο σε μία από τις 3 κατηγορίες [entailment/neutral/contradiction].

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "roberta-large-mnli")
classifier("A soccer game with multiple males playing. Some men are playing a sport.")
## [{'label': 'ENTAILMENT', 'score': 0.98}]
```

Ένα άλλο task ταξινόμησης αφορά την αξιολόγηση του κατά πόσο ένα κείμενο είναι γραμματικά ορθό (acceptable) ή όχι (unacceptable):

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "textattack/distilbert-base-uncased-CoLA")
classifier("I will walk to home when I went through the bus.")
##  [{'label': 'unacceptable', 'score': 0.95}]
```

## Σύνολο δεδομένων Yelp polarity

Κατεβάζουμε το [Yelp Polarity](https://huggingface.co/datasets/yelp_polarity) dataset το οποίο περιέχει reviews που εκφράζουν συναισθήματα πελατών για εστιατόρια. Τα reviews αυτά χωρίζονται σε κατηγορίες, και ο σκοπός μας είναι να κατηγοριοποιήσουμε νέα reviews στις σωστές κατηγορίες.

In [ ]:
from datasets import load_dataset

# insert your code here
dataset = load_dataset("fancyzhx/yelp_polarity")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for fancyzhx/yelp_polarity contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/fancyzhx/yelp_polarity
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `dataset

Επειδή το σύνολο δεδομένων του Yelp Polarity περιέχει πολλά δείγματα, προκειμένου να επιταχύνουμε τη διαδικασία του fine-tuning συστήνουμε να διατηρήσετε 300 δείγματα από το train set και 300 δείγματα από το test set.

Ελέγξτε τον αριθμό κατηγοριών που υπάρχουν συνολικά στο train και το test set και διατηρήστε ισορροπημένο αριθμό δειγμάτων ανά κατηγορία για τα σύνολα αυτά κατά την επιλογή των 300 δειγμάτων.

In [ ]:
# insert your code here
import pandas as pd
from sklearn.utils import resample

# Διατήρηση 300 δειγμάτων από το train set και το test set
def balanced_sample(dataset, sample_size):
    # Μετατροπή σε DataFrame για ευκολότερο χειρισμό
    df = pd.DataFrame(dataset)

    # Έλεγχος του αριθμού κατηγοριών
    categories = df['label'].unique()

    # Υπολογισμός του αριθμού δειγμάτων ανά κατηγορία
    samples_per_category = sample_size // len(categories)

    # Επιλογή ισορροπημένων δειγμάτων
    sampled_dfs = []
    for category in categories:
        category_df = df[df['label'] == category]
        sampled_category_df = resample(category_df, n_samples=samples_per_category, random_state=42)
        sampled_dfs.append(sampled_category_df)

    # Συνένωση των δειγμάτων ανά κατηγορία
    balanced_df = pd.concat(sampled_dfs)

    return balanced_df

# Διατήρηση 300 δειγμάτων από το train set και το test set
train_sampled = balanced_sample(dataset['train'], 300)
test_sampled = balanced_sample(dataset['test'], 300)

# Εμφάνιση των ισορροπημένων δειγμάτων
print("Train set samples:")
print(train_sampled)
print("\nTest set samples:")
print(test_sampled)

Train set samples:
                                                     text  label
232835  This review is based on the fact I can't bring...      0
283841  Poor value for money.\nI would not have normal...      0
253581  Reasonably priced (for the strip, I guess) caf...      0
515687  Disgusting. Grilled Fish taco looked like cat ...      0
228771  under a new management.\n\nFood quality and qu...      0
...                                                   ...    ...
266690  Chompie's is a fun place to go in AZ.  My kids...      1
118914  Amazing dining experience!  The service was to...      1
46928   Took my oldest dog here today for a hurt paw. ...      1
283284  \""Go there and dip your french fries in mayon...      1
456729  That's better. We used our free comp ticket fr...      1

[300 rows x 2 columns]

Test set samples:
                                                    text  label
31791  I've starting getting shellac manicures on a r...      1
1808   Best Lo Mein anywhere i

In [ ]:
#train_dataset

# Language Models

Η προεπεξεργασία των κειμένων προηγείται της εισόδου τους στα γλωσσικά μοντέλα. Η διαδικασία αυτή επιτελείται μέσω των Tokenizers, τα οποία μετατρέπουν τα tokens εισόδου σε κατάλληλα IDs του λεξιλογίου προεκπαίδευσης, κι έτσι μετατρέπουν το κείμενο σε μορφή που μπορεί να επεξεργαστεί κάποιο μοντέλο Transformer. Η βιβλιοθήκη Huggingface προσφέρει εύκολες και high-level υλοποιήσεις tokenization, τις οποίες συστήνουμε να ακολουθήσετε στη συνέχεια.

Συγκεκριμένα, αρχικοποιούμε τη διαδικασία του tokenization με χρήση του AutoTokenizer. Επιλέγοντας τη μέθοδο from_pretrained λαμβάνουμε έναν tokenizer που αποκρίνεται στην αρχιτεκτονική του μοντέλου που επιθυμούμε να χρησιμοποιήσουμε, παρέχοντας συμβατό tokenization.

Περισσότερες πληροφορίες για το AutoTokenization μπορείτε να βρείτε εδώ:
https://huggingface.co/docs/transformers/model_doc/auto

Αναφορικά με το μοντέλο BERT το οποίο διδαχθήκατε στο εργαστήριο, μπορείτε να δείτε τη διαδικασία [του tokenization και της αρχικοποίησης του μοντέλου](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertTokenizer):

```
from transformers import AutoTokenizer, BertModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
```

Στα πλαίσια της άσκησης καλείστε να επιτελέσετε την παραπάνω διαδικασία με *κάποιο άλλο μοντέλο της επιλογής σας από το Huggingface* που να υποστηρίζει τον AutoTokenizer. Το pre-trained μοντέλο που θα επιλέξετε θα πρέπει να διαθέτει υλοποίηση με sequence classification head (κατ αναλογία της μεθόδου BertForSequenceClassification).

Στο επόμενο κελί, φορτώστε το επιλεχθέν μοντέλο με τον αντίστοιχο tokenizer.

(Αγνοήστε πιθανά warnings της μορφής Some weights of the model checkpoint at xxx were not used when initializing...)

In [ ]:
# insert your code here
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Αρχικοποίηση του tokenizer και του μοντέλου
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenizer και μοντέλο έχουν αρχικοποιηθεί επιτυχώς.


Σας παρέχουμε τη συνάρτηση που πραγματοποιεί το tokenization καλώντας τον tokenizer που επιλέξατε. Εφαρμόστε το τόσο στο train, όσο και στο test set.

In [ ]:
from datasets import load_dataset, Dataset
from transformers import DataCollatorWithPadding

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# insert your code here
# Tokenization των συνόλων δεδομένων
train_sampled = Dataset.from_pandas(train_sampled)
test_sampled = Dataset.from_pandas(test_sampled)
train_tokenized = train_sampled.map(tokenize_function, batched=True)
test_tokenized = test_sampled.map(tokenize_function, batched=True)

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
#Data collation involves dynamically padding sequences to the maximum length within each batch during the data loading process.

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Τυπώνοντας το train ή το test set, θα δείτε δύο επιπλέον πεδία 'input_ids' και 'attention_mask'. Βεβαιωθείτε ότι υπάρχουν, άρα και το tokenization έχει επιτευχθεί.

In [ ]:
# Print a sample from the train_tokenized dataset
print(train_tokenized[0])

# Print a sample from the test_tokenized dataset
print(test_tokenized[0])


{'text': "This review is based on the fact I can't bring my camera in here. I was very disappointed in that. I just wanted to take some great shots. Bummer.", 'label': 0, '__index_level_0__': 232835, 'input_ids': [101, 2023, 3319, 2003, 2241, 2006, 1996, 2755, 1045, 2064, 1005, 1056, 3288, 2026, 4950, 1999, 2182, 1012, 1045, 2001, 2200, 9364, 1999, 2008, 1012, 1045, 2074, 2359, 2000, 2202, 2070, 2307, 7171, 1012, 26352, 5017, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
#train_dataset

## Χρήση του PyTorch Trainer για fine-tuning

Η κλάση [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) έχει βελτιστοποιηθεί από τους δημιουργούς του Huggingface παρέχοντας πολλές διευκολύνσεις και λιγότερη 'χεράτη' δουλειά. Προτείνουμε να τη χρησιμοποιήσετε ως εναλλακτική του να γράψετε το δικό σας training loop.
Καθώς η Trainer δεν τεστάρει αυτόματα την επίδοση του εκάστοτε μοντέλου κατά την εκπαίδευση, παρέχουμε κατάλληλη συνάρτηση προκειμένου να αποτιμάται το accuracy του μοντέλου σε κάθε εποχή.

In [ ]:
import numpy as np
import evaluate
import torch
from tqdm import tqdm
from transformers import pipeline

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Η κλάση [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments) περιέχει όλες τις υπερπαραμέτρους με τις οποίες μπορείτε να πειραματιστείτε κατά τη διαδικασία fine-tuning.


Καλείστε να πειραματιστείτε με διαφορετικές υπερπαραμέτρους όπως το learning rate, batch size κλπ, καθώς επίσης και να ορίσετε optimizer και scheduler για το fine-tuning. Προτείνουμε να εκτελέσετε fine-tuning για μικρό αριθμό εποχών (άλλωστε το μοντέλο είναι ήδη προεκπαιδευμένο).

1. Θα μας δώσετε σε markdown ένα πινακάκι με διαφορετικές υπερπαραμέτρους που δοκιμάσατε και το accuracy που πετύχατε στην τελευταία εποχή.

2. Βάσει των πειραματισμών, πώς επηρεάζουν διαφορετικές υπερπαράμετροι όπως το learning rate και το batch size το fine-tuning του μοντέλου που επιλέξατε? Σχολιάστε και αναλύστε.

In [ ]:
from transformers import TrainingArguments, Trainer, AdamW, get_linear_schedule_with_warmup

args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch",learning_rate=4e-5, per_device_train_batch_size=8)


# insert your code here

# optimizer
optimizer = AdamW(model.parameters(), lr=args.learning_rate)
# scheduler
num_training_steps = len(train_tokenized) * args.num_train_epochs
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)
# etc

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Έτρεξα τον κώδικα με τις παρακάτω υπερπαραμέτρους και παίρνουμε το παρακάτω markdown πινακάκι:

```
| Learning Rate | Batch Size | Accuracy |
|---------------|------------|----------|
| 2e-5          | 8          | 0.896    |
| 3e-5          | 8          | 0.896    |
| 1e-5          | 8          | 0.886    |
| 2e-5          | 16         | 0.886    |
| 3e-5          | 16         | 0.913    |
| 4e-5          | 8          | 0.903    |


```

### Παρατηρήσεις:

1. Μέτρια έως υψηλότερα learning rates (3e-5, 4e-5) φαίνεται να αποδίδουν καλύτερα.
2. Η διαφοροποίηση στο batch size παίζει ρόλο, αλλά εν τέλει σημασία έχει ο σωστότερος συνδυασμός batch size και learning rate (3e-5).
3. Το batch size = 16 με learning rate 3e-5 έχει το καλύτερο accuracy (0.914).

In [ ]:

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    compute_metrics=compute_metrics,
)

Then fine-tune your model by calling [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train):

In [ ]:
trained_model=trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.233598,0.890000
2,No log,0.986981,0.910000
3,No log,0.918273,0.903333


# Μέρος Β: Χρήση fine-tuned μοντέλων σε νέα tasks

Στο κομμάτι αυτό της εργασίας δε χρειάζεται να πραγματοποιήσετε εκπαίδευση σε γλωσσικά μοντέλα. Αντιθέτως, θα εκμεταλλευτούμε τις δυνατότητες του transfer learning για να αντιμετωπίσουμε πιο πολύπλοκα γλωσσικά task, ανάγοντάς τα σε κλασικά task όπως είναι το text classification, natural language inference, question answering και άλλα.

Για παράδειγμα, fine-tuned μοντέλα για [text classification](https://huggingface.co/tasks/text-classification) εξυπηρετούν tasks όπως:

- Είναι δύο προτάσεις η μία παράφραση της άλλης? [Paraphrase/No Paraphrase]
- Συνεπάγεται η πρόταση Χ την πρόταση Υ? [Entail/Neutral/Contradict]
- Είναι η δοθείσα πρόταση γραμματικά ορθή? [Acceptable/Unacceptable]

## B1. Piqa dataset

Το [Piqa dataset](https://huggingface.co/datasets/piqa) περιλαμβάνει προτάσεις οι οποίες ελέγχουν το βαθμό στον οποίο τα language models έχουν κοινή γνώση (commonsense). Συγκεκριμένα, αποτελείται από προτάσεις και πιθανά endings, τα οποία απαιτούν commonsense γνώση για να συμπληρωθούν.

Για παράδειγμα, έχοντας την πρόταση "When boiling butter, when it's ready, you can" υπάρχουν δύο υποψήφια endings:
- "Pour it onto a plate"
- "Pour it into a jar"

Ένας άνθρωπος μπορεί να συμπεράνει ότι η δεύτερη πρόταση αποτελεί ένα πιο κατάλληλο ending, αφού το λιωμένο βούτυρο είναι υγρό, άρα το βάζο είναι ένα καταλληλότερο δοχείο σε σχέση με το πιάτο.

Για λόγους επιτάχυνσης επιλέξτε ένα τυχαίο υποσύνολο 100 δειγμάτων από το Piqa.

In [ ]:
# # insert your code here (load dataset)
from datasets import load_dataset
import random
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch
import pandas as pd
import numpy as np

# Load the PIQA dataset
dataset = load_dataset("piqa")

# Set the random seed for reproducibility
random.seed(42)

# Select a random subset of 100 examples from the training set
subset_indices = random.sample(range(len(dataset['train'])), 100)
subset = dataset['train'].select(subset_indices)

# Convert the subset to a pandas DataFrame
subset = pd.DataFrame(subset)

# Display the first few rows of the subset
print(subset.head())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for piqa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/piqa
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/16113 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3084 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1838 [00:00<?, ? examples/s]

                                 goal  \
0     To make strawberries less sour,   
1  How can I dough in a circle shape?   
2               Make an outdoor fort.   
3                     Create a spear.   
4                          chopsticks   

                                                sol1  \
0                      sprinkle with a bit of sugar.   
1  Use the top of a full glass and press into dough.   
2     Use a hula hoop to hang a bed sheet from tree.   
3                Scotch tape a knife to large stick.   
4                                    can play piano    

                                                sol2  label  
0                       drop a bit of water on them.      0  
1  Use the top of an empty glass and press into d...      1  
2  Use a hula hoop to hang a shower curtain from ...      0  
3                  Duct tape a knife to large stick.      1  
4                              can play a videogame       0  


Μπορούμε να θεωρήσουμε το παραπάνω σενάριο σαν ένα πρόβλημα πολλαπλής επιλογής, όπου υπάρχουν δύο πιθανές εναλλακτικές για το ending της πρότασης. Συνεπώς, αξιοποιώντας σχετικά μοντέλα μπορούμε να επιλύσουμε την επιλογή του ending δοθείσας της πρότασης.

Καλείστε λοιπόν να καταγράψετε το accuracy πρόβλεψης endings για κάθε πρόταση με χρήση γλωσσικών μοντέλων. Για λόγους σύγκρισης χρησιμοποιήστε τουλάχιστον 5 κατάλληλα μοντέλα.

In [ ]:
# insert your code here (models)
from transformers import (
    BertTokenizer, BertForSequenceClassification,
    GPT2Tokenizer, GPT2ForSequenceClassification,
    DistilBertTokenizer, DistilBertForSequenceClassification,
    AlbertTokenizer, AlbertForSequenceClassification,
    XLNetTokenizer, XLNetForSequenceClassification
)

# Define the models and their corresponding tokenizers
models = [
    ("bert-base-uncased", BertTokenizer, BertForSequenceClassification),
    ("distilbert-base-uncased", DistilBertTokenizer, DistilBertForSequenceClassification),
    ("albert-base-v2", AlbertTokenizer, AlbertForSequenceClassification),
    ("xlnet-base-cased", XLNetTokenizer, XLNetForSequenceClassification),
    ("gpt2", GPT2Tokenizer, GPT2ForSequenceClassification),
]

In [ ]:
# insert your code here (function for ending prediction)
def predict_ending(model_name, tokenizer_class, model_class, dataset):
    # Load the tokenizer and model
    tokenizer = tokenizer_class.from_pretrained(model_name)
    model = model_class.from_pretrained(model_name)

    # Prepare the input data for the model
    predictions = []
    for i, row in dataset.iterrows():
        context = row['goal']  # The context or initial part of the sentence
        ending1 = row['sol1']  # First possible ending
        ending2 = row['sol2']  # Second possible ending

        # Encode the input for the model
        inputs1 = tokenizer(context + " " + ending1, return_tensors='pt')
        inputs2 = tokenizer(context + " " + ending2, return_tensors='pt')

        # Get the model's prediction for each ending
        outputs1 = model(**inputs1)
        outputs2 = model(**inputs2)

        # Use the logits from the model's output
        logits1 = outputs1.logits
        logits2 = outputs2.logits

        # Compare the logits to decide the predicted ending
        pred = 0 if logits1.argmax() > logits2.argmax() else 1
        predictions.append(pred)

    return predictions

def compute_accuracy(predictions, labels):
    return np.mean(np.array(predictions) == np.array(labels))

In [ ]:
# Extract the true labels from the dataset
true_labels = [0 if row['label'] == 'sol1' else 1 for _, row in subset.iterrows()]

# Evaluate each model and print the results
results = {}
for model_name, tokenizer_class, model_class in models:
    print(f"Evaluating {model_name}...")
    predictions = predict_ending(model_name, tokenizer_class, model_class, subset)
    accuracy = compute_accuracy(predictions, true_labels)
    results[model_name] = accuracy

# Print the accuracy results
for model_name, accuracy in results.items():
    print(f"Model: {model_name}, Accuracy: {accuracy:.4f}")

Evaluating bert-base-uncased...


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: bert-base-uncased, Accuracy: 1.0000
Model: distilbert-base-uncased, Accuracy: 1.0000
Model: albert-base-v2, Accuracy: 0.9400
Model: xlnet-base-cased, Accuracy: 1.0000
Model: gpt2, Accuracy: 1.0000


## B2. Truthful QA

### Sentence Transformers

Οι sentence transformers χρησιμοποιούνται για να δημιουργήσουν embeddings προτάσεων, δηλαδή διανυσματικές αναπαραστάσεις των προτάσεων αυτών σε ένα διανυσματικό χώρο. Χάρη στον τρόπο που έχουν προεκπαιδευτεί, έχουν την ικανότητα να τοποθετούν νοηματικά όμοιες προτάσεις κοντά τη μία στην άλλη, ενώ απομακρύνουν νοηματικά μακρινές προτάσεις. Έτσι, χάρη στις αναπαραστάσεις που λαμβάνουμε από τα sentence embeddings μπορούμε να αξιολογήσουμε σε τι βαθμό δύο προτάσεις είναι κοντά ή μακριά νοηματικά.

Η σύγκριση των διανυσματικών αναπαραστάσεων μπορεί να γίνει κλασικά μέσω μεθόδων όπως το consine similarity, με μεγαλύτερες τιμές μεταξύ διανυσμάτων να σηματοδοτούν πιο όμοια διανύσματα, άρα και πιο όμοιες προτάσεις. Δίνουμε για το λόγο αυτό μια συνάρτηση υπολογισμού του cosine similarity.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine_similarity(feature_vec_1, feature_vec_2):
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]

Για παράδειγμα, εκτελέστε το ακόλουθο κελί, το οποίο δίνει μια τιμή ομοιότητας στο διάστημα [0, 1] για δύο προτάσεις ("This is an example sentence", "Each sentence is converted"). Μπορείτε ακόμα να δοκιμάσετε να εκτελέσετε το ακόλουθο κελί για διαφορετικές προτάσεις της επιλογής σας, που μπορεί να είναι όμοιες ή πολύ διαφορετικές μεταξύ τους, και να παρατηρήσετε τις αλλαγές τιμών του cosine similarity.

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["Ayoub El Kaabi is a center forward", "Philip is a magician"]

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences)

get_cosine_similarity(embeddings[0], embeddings[1])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


0.16399899

Για τη συνέχεια της άσκησης, καλείστε να επιλέξετε τουλάχιστον 6 διαφορετικά [μοντέλα για semantic similarity](https://huggingface.co/models?pipeline_tag=sentence-similarity&sort=downloads) από τους sentence transformers

### Μπορούν τα question answering μοντέλα να διαχωρίσουν αληθείς και ψευδείς προτάσεις?

Αυτό το ερώτημα θα το απαντήσουμε στο παρόν κομμάτι της άσκησης. Για το λόγο αυτό, φορτώνουμε το dataset [Truthful QA generation](https://huggingface.co/datasets/truthful_qa/viewer/generation/validation), το οποίο περιέχει τις εξής επιλογές:

- best answer
- correct answer
- incorrect answer

Πολλές φορές το best answer και το correct answer είναι ίδια ή έστω πολύ κοντινά νοηματικά. Σε αυτό το σημείο είναι που θα αξιοποιήσουμε το semantic similarity για να αξιολογήσουμε την ομοιότητα αυτή.

Φιλτράρουμε το dataset ώστε να περιέχονται 100 δείγματα συνολικά για λόγους επιτάχυνσης, εκ των οποίων καθένα θα πρέπει να περιέχει τουλάχιστον 2 correct answer. Θεωρούμε έτσι 4 υποψήφιες επιλογές:

1η επιλογή: best answer  
2η επιλογή: 1ο correct answer  
3η επιλογή: 2ο correct answer  
4η επιλογή: incorrect answer  

Οι επιλογές αυτές μαζί με την ερώτηση δίνονται σε ένα μοντέλο πολλαπλής επιλογής σαν αυτά που χρησιμοποιήθηκαν στο ερώτημα Β1. Μπορείτε να θεωρήσετε τα ίδια μοντέλα και να τα επεκτείνετε σε 4 υποψήφιες απαντήσεις.  

Το semantic similarity θα επηρεάσει το τι θεωρούμε βέλτιστα σωστή απάντηση, άρα και το accuracy. Συγκεκριμένα, θα λάβουμε διανυσματικές αναπαραστάσεις για το best answer και τα 2 correct answer που έχουν δοθεί ως υποψήφιες επιλογές μέσω κάποιου semantic similarity μοντέλου. Σε περίπτωση λοιπόν που το μοντέλο πολλαπλής επιλογής προβλέψει ένα εκ των correct answer, και η ομοιότητά τους σε σχέση με το best model ξεπερνάει ένα προεπιλεγμένο κατώφλι ομοιότητας, η απάντηση θεωρείται βέλτιστα σωστή. Θέτουμε λοιπόν κατώφλι ομοιότητας το 0.95.

Για παράδειγμα, έστω ότι το μοντέλο πολλαπλής επιλογής μεταξύ των υποψηφίων [best, 1st correct, 2nd correct, incorrect] επιλέγει το δεύτερο στοιχείο, δηλαδή το 1st correct, και δεδομένου ότι το cosine similarity μεταξύ των embeddings του best και του 1st correct είναι > 0.95, τότε θεωρούμε ότι η απάντηση είναι βέλτιστα σωστή, και συνυπολογίζεται θετικά στο accuracy.

Καλείστε λοιπόν να γράψετε μια συνάρτηση που να υπολογίζει το accuracy εύρεσης βέλτιστα σωστών απαντήσεων ανάμεσα στις υποψήφιες απαντήσεις, εξετάζοντας τουλάχιστον 6 semantic similarity μοντέλα καθώς επίσης και τα μοντέλα που επιλέξατε στο ερώτημα Β1.


In [ ]:
import logging
from datasets import load_dataset
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import torch
from transformers import BertTokenizer, BertForSequenceClassification, \
                         RobertaTokenizer, RobertaForSequenceClassification, \
                         DistilBertTokenizer, DistilBertForSequenceClassification, \
                         AlbertTokenizer, AlbertForSequenceClassification, \
                         XLNetTokenizer, XLNetForSequenceClassification


# Set logging level to ERROR to suppress warnings
logging.basicConfig(level=logging.ERROR)

# Φόρτωση του TruthfulQA dataset
dataset = load_dataset("truthful_qa", "generation")

# Επιλέγουμε ένα υποσύνολο του dataset για ταχύτητα
subset = dataset['validation'].shuffle(seed=42).select(range(100))


In [ ]:
# insert your code here (load models for semantic similarity and QA)

# Φόρτωση Sentence Transformers μοντέλων με τα ονόματά τους
similarity_models = [
    ("all-mpnet-base-v2", SentenceTransformer('sentence-transformers/all-mpnet-base-v2')),
    ("paraphrase-MiniLM-L6-v2", SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')),
    ("paraphrase-MiniLM-L12-v2", SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L12-v2')),
    ("paraphrase-xlm-r-multilingual-v1", SentenceTransformer('sentence-transformers/paraphrase-xlm-r-multilingual-v1')),
    ("stsb-roberta-large", SentenceTransformer('sentence-transformers/stsb-roberta-large')),
    ("distiluse-base-multilingual-cased-v2", SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2'))
]

# Ορισμός των QA μοντέλων και των αντίστοιχων tokenizers
qa_models = [
    ("bert-base-uncased", BertForSequenceClassification, BertTokenizer),
    ("distilbert-base-uncased", DistilBertForSequenceClassification, DistilBertTokenizer),
    ("albert-base-v2", AlbertForSequenceClassification, AlbertTokenizer),
    ("xlnet-base-cased", XLNetForSequenceClassification, XLNetTokenizer),
    ("roberta-base", RobertaForSequenceClassification, RobertaTokenizer),
]

In [ ]:
# insert your code here (function for optimal correct answers & semantic similarity)

# Συνάρτηση υπολογισμού cosine similarity
def get_cosine_similarity(vec1, vec2):
    return cosine_similarity(vec1.reshape(1, -1), vec2.reshape(1, -1))[0][0]

# Συνάρτηση υπολογισμού accuracy για ένα συγκεκριμένο QA μοντέλο
def calculate_accuracy_for_qa_model(dataset, similarity_models, qa_model, threshold=0.95):
    accuracy_results = {}
    qa_model_name, model_class, tokenizer_class = qa_model


    for sim_model_name, sim_model in similarity_models:
        correct_predictions = 0
        for data in dataset:
            question = data["question"]
            best_answer = data["best_answer"]
            correct_answers = data["correct_answers"][:2]  # Χρησιμοποιούμε τις πρώτες 2 σωστές απαντήσεις
            incorrect_answer = data["incorrect_answers"][0]  # Χρησιμοποιούμε την πρώτη λανθασμένη απάντηση

            candidate_answers = [best_answer] + correct_answers + [incorrect_answer]
            # Φόρτωση embeddings των απαντήσεων με το current similarity model
            embeddings = sim_model.encode(candidate_answers)
            best_embedding = embeddings[0]

            # Εύρεση της πιο παρόμοιας απάντησης με το best_answer
            similarities = [get_cosine_similarity(best_embedding, emb) for emb in embeddings[1:]]

            # Εύρεση της πρόβλεψης από το QA μοντέλο
            tokenizer = tokenizer_class.from_pretrained(qa_model_name)
            model = model_class.from_pretrained(qa_model_name)

            inputs = tokenizer([question] + candidate_answers, padding=True, return_tensors="pt")
            outputs = model(**inputs)
            predictions = torch.argmax(outputs.logits, dim=-1) #Extracts the index of the maximum value along the last dimension of the logits tensor, representing the predicted class.
            predicted_answer = candidate_answers[predictions[0]] #Retrieves the predicted answer corresponding to the predicted class index.

            # Υπολογισμός ομοιότητας και απόφαση αν η πρόβλεψη είναι βέλτιστα σωστή
            if predicted_answer in correct_answers:
                similarity = get_cosine_similarity(best_embedding, sim_model.encode(predicted_answer).reshape(1, -1))
                if similarity > threshold:
                    correct_predictions += 1

        accuracy = correct_predictions / len(dataset)
        accuracy_results[f"{sim_model_name} + {qa_model_name}"] = accuracy

    return accuracy_results

## Παρακάτω φαίνονται ένα-ένα τα μοντέλα:

### distilbert-base-uncased

In [ ]:
# Παράδειγμα χρήσης της συνάρτησης για ένα συγκεκριμένο QA μοντέλο
qa_model =  ("distilbert-base-uncased", DistilBertForSequenceClassification, DistilBertTokenizer)
accuracy_results = calculate_accuracy_for_qa_model(subset, similarity_models, qa_model)
for model_combination, accuracy in accuracy_results.items():
    print(f"{model_combination}: {accuracy:.2f}")

```
all-mpnet-base-v2 + distilbert-base-uncased: 0.96
paraphrase-MiniLM-L6-v2 + distilbert-base-uncased: 0.95
paraphrase-MiniLM-L12-v2 + distilbert-base-uncased: 0.95
paraphrase-xlm-r-multilingual-v1 + distilbert-base-uncased: 0.97
stsb-roberta-large + distilbert-base-uncased: 0.96
distiluse-base-multilingual-cased-v2 + distilbert-base-uncased: 0.95
```

### albert-base-v2

In [ ]:
# Παράδειγμα χρήσης της συνάρτησης για ένα συγκεκριμένο QA μοντέλο
qa_model =  ("albert-base-v2", AlbertForSequenceClassification, AlbertTokenizer)
accuracy_results = calculate_accuracy_for_qa_model(subset, similarity_models, qa_model)
for model_combination, accuracy in accuracy_results.items():
    print(f"{model_combination}: {accuracy:.2f}")

```
all-mpnet-base-v2 + albert-base-v2: 0.95
paraphrase-MiniLM-L6-v2 + albert-base-v2: 0.96
paraphrase-MiniLM-L12-v2 + albert-base-v2: 0.97
paraphrase-xlm-r-multilingual-v1 + albert-base-v2: 0.95
stsb-roberta-large + albert-base-v2: 0.96
distiluse-base-multilingual-cased-v2 + albert-base-v2: 0.95
```


### xlnet-base-cased

In [ ]:
# Παράδειγμα χρήσης της συνάρτησης για ένα συγκεκριμένο QA μοντέλο
qa_model =  ("xlnet-base-cased", XLNetForSequenceClassification, XLNetTokenizer)
accuracy_results = calculate_accuracy_for_qa_model(subset, similarity_models, qa_model)
for model_combination, accuracy in accuracy_results.items():
    print(f"{model_combination}: {accuracy:.2f}")

```
all-mpnet-base-v2 + xlnet-base-cased: 0.97
paraphrase-MiniLM-L6-v2 + xlnet-base-cased: 0.95
paraphrase-MiniLM-L12-v2 + xlnet-base-cased: 0.95
paraphrase-xlm-r-multilingual-v1 + xlnet-base-cased: 0.95
stsb-roberta-large + xlnet-base-cased: 0.96
distiluse-base-multilingual-cased-v2 + xlnet-base-cased: 0.96
```

### roberta-base

In [ ]:
# Παράδειγμα χρήσης της συνάρτησης για ένα συγκεκριμένο QA μοντέλο
qa_model =  ("roberta-base", RobertaForSequenceClassification, RobertaTokenizer)
accuracy_results = calculate_accuracy_for_qa_model(subset, similarity_models, qa_model)
for model_combination, accuracy in accuracy_results.items():
    print(f"{model_combination}: {accuracy:.2f}")

```
all-mpnet-base-v2 + roberta-base: 0.97
paraphrase-MiniLM-L6-v2 + roberta-base: 0.96
paraphrase-MiniLM-L12-v2 + roberta-base: 0.95
paraphrase-xlm-r-multilingual-v1 + roberta-base: 0.96
stsb-roberta-large + roberta-base: 0.95
distiluse-base-multilingual-cased-v2 + roberta-base: 0.95
```

## Β3. Winogrande dataset

Το [Winogrande dataset](https://huggingface.co/datasets/winogrande) αποτελείται από προτάσεις που μία λέξη τους έχει αφαιρεθεί και δίνονται δύο πιθανές επιλογές συμπλήρωσης του κενού. Για παράδειγμα, δοθείσας της πρότασης "John moved the couch from the garage to the backyard to create space. The _ is small.", υπάρχουν δύο πιθανές εναλλακτικές:

- "garage"
- "backyard"

Η δυσκολία της συμπλήρωσης έγκειται στο ότι και οι δύο λέξεις αναφέρονται στην πρόταση, οπότε το μοντέλο θα πρέπει να διαθέτει υψηλές δυνατότητες κατανόησης γλώσσας προκειμένου να επιλέξει μια νοηματικά σωστή συμπλήρωση.

Για λόγους επιτάχυνσης, επιλέξτε ένα τυχαίο υποσύνολο 100 δειγμάτων από το training set του Winogrande.


In [ ]:
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import random
import torch

# Load the dataset
dataset = load_dataset("allenai/winogrande", "winogrande_xl")

# Select a random subset of 100 samples from the training set
random.seed(42)  # For reproducibility
train_subset = dataset['train'].shuffle(seed=42).select(range(100))

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for allenai/winogrande contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/allenai/winogrande
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/40398 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1767 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1267 [00:00<?, ? examples/s]

Με κατάλληλο [μετασχηματισμό](https://huggingface.co/DeepPavlov/roberta-large-winogrande) της παραπάνω εισόδου (πρόταση με κενό και δύο επιλογές συμπλήρωσης), καλείστε να καταγράψετε το accuracy σχετικών μοντέλων που επιλύουν το πρόβλημα, συγκρίνοντας το predicted label με το πραγματικό label (1: πρώτη επιλογή, 2: δεύτερη επιλογή). Ουσιαστικά θα πρέπει να αναγάγετε το παραπάνω πρόβλημα σε κάποιο πιο κλασικό πρόβλημα της επεξεργασίας φυσικής γλώσσας.

Δοκιμάστε τουλάχιστον 3 κατάλληλα μοντέλα από το Huggingface για να προσεγγίσετε το πρόβλημα του Winogrande. Προτείνουμε τη χρήση pipelines για τη διευκόλυνσή σας.

- Πρώτα έχω τρέξει το πρώτο (και καλύτερο μοντέλο "DeepPavlov/roberta-large-winogrande") τυπώνοντας τα "scores".

- Κι έπειτα έχω τρέξει τα επόμενα δύο μοντέλα (χωρίς την εκτύπωση των "scores").

In [ ]:
# insert your code here (load models)
# The first model (which is fine-tuned in the dataset)
# The 2nd and 3rd models are in the final cell.
model_names = [
    "DeepPavlov/roberta-large-winogrande",
]

# Function to transform the dataset format
def transform_example(example):
    # Split the sentence around the underscore
    sentence_parts = example['sentence'].split('_')
    sentence1 = sentence_parts[0]
    sentence2 = sentence_parts[1]

    # Create the two transformed examples
    transformed = [
        {'sentence1': sentence1, 'sentence2': example['option1'] + sentence2, 'label': example['answer'] == '1', 'option': 1},
        {'sentence1': sentence1, 'sentence2': example['option2'] + sentence2, 'label': example['answer'] == '2', 'option': 2}
    ]

    return transformed


# Apply transformation to all examples in the subset
transformed_data = []
for example in train_subset:
    transformed_data.extend(transform_example(example))

print(transformed_data)

[{'sentence1': "The phone of Donald is a lot better than Adam's because ", 'sentence2': 'Donald paid extra for his phone.', 'label': True, 'option': 1}, {'sentence1': "The phone of Donald is a lot better than Adam's because ", 'sentence2': 'Adam paid extra for his phone.', 'label': False, 'option': 2}, {'sentence1': 'Dennis was buying more books while Donald was buying more video games because ', 'sentence2': 'Dennis was more studious.', 'label': True, 'option': 1}, {'sentence1': 'Dennis was buying more books while Donald was buying more video games because ', 'sentence2': 'Donald was more studious.', 'label': False, 'option': 2}, {'sentence1': 'Jessica sneezed more than Carrie was sneezing because there was more dust in the room of ', 'sentence2': 'Jessica .', 'label': True, 'option': 1}, {'sentence1': 'Jessica sneezed more than Carrie was sneezing because there was more dust in the room of ', 'sentence2': 'Carrie .', 'label': False, 'option': 2}, {'sentence1': 'When it comes to trave

In [ ]:
# insert your code here (create pipelines)
pipelines = []
for model_name in model_names:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, return_all_scores=True)
    pipelines.append(pipe)

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/820 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
# insert your code here (function for predicting best fill)
def predict_best_fill(pipeline, example1, example2):
    input1 = example1['sentence1'] + example1['sentence2']
    input2 = example2['sentence1'] + example2['sentence2']

    results = pipeline([input1, input2])
    print(results) # I print the scores for the model "DeepPavlov/roberta-large-winogrande"

    score1 = results[0][0]['score'] if results[0][0]['label'] == 'LABEL_1' else results[0][1]['score']
    score2 = results[1][0]['score'] if results[1][0]['label'] == 'LABEL_1' else results[1][1]['score']

    return example1['option'] if score1 > score2 else example2['option']

In [ ]:
# Calculate accuracy for each model
def calculate_accuracy(pipeline, data):
    correct_predictions = 0
    total_predictions = len(data) // 2

    for i in range(0, len(data), 2):
        example1 = data[i]
        example2 = data[i + 1]
        prediction = predict_best_fill(pipeline, example1, example2)
        actual_label = 1 if example1['label'] else 2
        if prediction == actual_label:
            correct_predictions += 1

    return correct_predictions / total_predictions

# Evaluate each pipeline and print the accuracy
for i, pipe in enumerate(pipelines):
    accuracy = calculate_accuracy(pipe, transformed_data)
    print(f"Model {model_names[i]} Accuracy: {accuracy * 100:.2f}%")

[[{'label': 'False', 'score': 0.30675527453422546}, {'label': 'True', 'score': 0.6932447552680969}], [{'label': 'False', 'score': 0.7141005992889404}, {'label': 'True', 'score': 0.28589946031570435}]]
[[{'label': 'False', 'score': 0.6489669680595398}, {'label': 'True', 'score': 0.3510330617427826}], [{'label': 'False', 'score': 0.9280688166618347}, {'label': 'True', 'score': 0.07193122059106827}]]
[[{'label': 'False', 'score': 0.0001775623532012105}, {'label': 'True', 'score': 0.9998224377632141}], [{'label': 'False', 'score': 0.9871370792388916}, {'label': 'True', 'score': 0.012862888164818287}]]
[[{'label': 'False', 'score': 0.02181207202374935}, {'label': 'True', 'score': 0.9781879186630249}], [{'label': 'False', 'score': 0.9580106139183044}, {'label': 'True', 'score': 0.04198942333459854}]]
[[{'label': 'False', 'score': 0.9978554844856262}, {'label': 'True', 'score': 0.0021445450838655233}], [{'label': 'False', 'score': 3.324905628687702e-05}, {'label': 'True', 'score': 0.999966740

- Τα επόμενα δύο μοντέλα (χωρίς την εκτύπωση των "scores").

In [ ]:
# The following 2 models (without printing the scores)
model_names = [
    "roberta-large",
    "bert-large-uncased"
]

pipelines = []
for model_name in model_names:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, return_all_scores=True)
    pipelines.append(pipe)

# predict_best_fill (without printing the scores)
def predict_best_fill(pipeline, example1, example2):
    input1 = example1['sentence1'] + example1['sentence2']
    input2 = example2['sentence1'] + example2['sentence2']

    results = pipeline([input1, input2])

    score1 = results[0][0]['score'] if results[0][0]['label'] == 'LABEL_1' else results[0][1]['score']
    score2 = results[1][0]['score'] if results[1][0]['label'] == 'LABEL_1' else results[1][1]['score']

    return example1['option'] if score1 > score2 else example2['option']

# Evaluate each pipeline and print the accuracy
for i, pipe in enumerate(pipelines):
    accuracy = calculate_accuracy(pipe, transformed_data)
    print(f"Model {model_names[i]} Accuracy: {accuracy * 100:.2f}%")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model roberta-large Accuracy: 42.00%
Model bert-large-uncased Accuracy: 44.00%
